In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import requests

In [2]:
def load_data(file_path, url_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
    else:
        d,i,offset_size = [],0,100000
        k = offset_size
        while k == offset_size:
            if i > 10:
                break
            r=requests.get(
                url = url_path, 
                params={
                    '$limit':offset_size, 
                    '$offset':offset_size*i
                }
            )
    
            as_df = pd.DataFrame(r.json())
            k = len(as_df)
            i += 1
            d.append(as_df)
    
        df = pd.concat(d)
        df.to_csv(file_path)
        df.head(10)
    return df

In [3]:
tree_census_df =load_data('tree_census/flatfiles/tree_census_15.csv', 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json')

In [4]:
air_quality_df = load_data('data/Air_Quality_20241206.csv', 'https://data.cityofnewyork.us/resource/c3uy-2p5r.json')

In [5]:
green_streets_df = load_data('data/Greenstreets.csv', 'https://data.cityofnewyork.us/resource/mk9u-qu7i.json')

In [6]:
property_valuation_df = load_data('data/Property_Valuation_and_Assessment_Data_20241125.csv', 'https://data.cityofnewyork.us/resource/yjxr-fw8i.json')

# Tree Census Data Sanitation 

In [7]:
cols = [
    'tree_id','block_id','the_geom','zipcode','cb_num','borocode','nta_name',
    'health','status','tree_dbh','spc_common','spc_latin','problems','sidewalk',
]

tree_census_tr = tree_census_df[cols]
tree_census_tr = tree_census_tr.loc[tree_census_df['status']=='Alive']
tree_census_tr = tree_census_tr.loc[~tree_census_tr['health'].isna()]
tree_census_tr['tree_dbh'] = tree_census_tr['tree_dbh'].astype(int)

tree_census_tr.head()

,tree_id,block_id,the_geom,zipcode,cb_num,borocode,nta_name,health,status,tree_dbh,spc_common,spc_latin,problems,sidewalk
0,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",11375,406,4,Forest Hills,Fair,Alive,3,red maple,Acer rubrum,NaN,NoDamage
1,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",11357,407,4,Whitestone,Fair,Alive,21,pin oak,Quercus palustris,Stones,Damage
2,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",11211,301,3,East Williamsburg,Good,Alive,3,honeylocust,Gleditsia triacanthos var. inermis,NaN,Damage
3,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",11211,301,3,East Williamsburg,Good,Alive,10,honeylocust,Gleditsia triacanthos var. inermis,Stones,Damage
4,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",11215,306,3,Park Slope-Gowanus,Good,Alive,21,American linden,Tilia americana,Stones,Damage


In [12]:
#make analysis at zipcode level...
cb_grp = tree_census_tr.groupby(['cb_num']).agg(
    tree_count=('tree_id', len), 
    tree_size=('tree_dbh', np.sum)
)

#avg tree size (diameter)
cb_grp['tree_size'] = cb_grp['tree_size']/cb_grp['tree_count']

#count trees per zip/health category (good, fair, bad), then pivot on the health cats
cb_grp_health_status = tree_census_tr.groupby(['cb_num','health'])[['tree_id']].count().reset_index()
cb_grp_health_status_piv = cb_grp_health_status.pivot(
    index='cb_num',
    columns='health', 
    values='tree_id'
)

#concat cb features
cb_grp_health_status_piv = cb_grp_health_status_piv.div(cb_grp['tree_count'], axis=0)
tree_cb_features = pd.concat((cb_grp_health_status_piv,cb_grp),axis=1)

tree_cb_features['Good'] = tree_cb_features['Good'].fillna(0.0)
tree_cb_features = tree_cb_features.rename(
    columns={
        'Good':'Tree Quality'
    }
)

tree_cb_features = tree_cb_features.reset_index()
tree_cb_features['cb_num'] = tree_cb_features['cb_num'].astype(str)
tree_cb_features = tree_cb_features[['cb_num', 'Tree Quality', 'tree_count']]
tree_cb_features.head()

/var/folders/wx/wz1pbs1903j_cfgqws400dz80000gn/T/ipykernel_70401/1516815439.py:2: FutureWarning: The provided callable <function sum at 0x11859a8b0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cb_grp = tree_census_tr.groupby(['cb_num']).agg(


,cb_num,Tree Quality,tree_count
0,101,0.764040,2297
1,102,0.730602,4833
2,103,0.764069,4709
3,104,0.824395,4419
4,105,0.673560,2031


In [15]:
cb_geometries = gpd.read_file("tree_census/flatfiles/Community Districts.geojson")
geo_cols = ['boro_cd','shape_area','geometry']
cb_geometries = cb_geometries[geo_cols].rename(
    columns={
        'boro_cd':'cb_num'
    }
)
tree_census_geoms = tree_cb_features.merge(
    cb_geometries, 
    how='inner', 
    on='cb_num'
)

tree_census_geoms['cb_acreage'] = tree_census_geoms['shape_area'].astype(np.float64)/43560
tree_census_geoms['Tree Density (per Acre)'] = tree_census_geoms['tree_count']/tree_census_geoms['cb_acreage']

tree_census_geoms.head()


,cb_num,Tree Quality,tree_count,shape_area,geometry,cb_acreage,Tree Density (per Acre)
0,101,0.764040,2297,41692711.732,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",957.132960,2.399876
1,102,0.730602,4833,37724014.4661,"MULTIPOLYGON (((-74.00915 40.7425, -74.00902 4...",866.024207,5.580675
2,103,0.764069,4709,46879905.4307,"MULTIPOLYGON (((-73.98878 40.73397, -73.98757 ...",1076.214542,4.375522
3,104,0.824395,4419,49310051.9139,"MULTIPOLYGON (((-73.99394 40.77318, -73.9937 4...",1132.003028,3.903700
4,105,0.673560,2031,43790039.865,"MULTIPOLYGON (((-73.97301 40.76428, -73.97141 ...",1005.280989,2.020331


# Air Quality Data Sanitation

In [17]:
air_quality_df.head()

,Unnamed: 0,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,0,827080,386,Ozone (O3),Mean,ppb,UHF34,104.0,Pelham - Throgs Neck,Summer 2022,2022-06-01T00:00:00.000,33.3
1,1,827061,386,Ozone (O3),Mean,ppb,UHF34,405.0,Ridgewood - Forest Hills,Summer 2022,2022-06-01T00:00:00.000,34.2
2,2,827067,386,Ozone (O3),Mean,ppb,UHF34,302.0,Central Harlem - Morningside Heights,Summer 2022,2022-06-01T00:00:00.000,30.9
3,3,827081,386,Ozone (O3),Mean,ppb,UHF34,103.0,Fordham - Bronx Pk,Summer 2022,2022-06-01T00:00:00.000,31.7
4,4,825967,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,104.0,Pelham - Throgs Neck,Summer 2022,2022-06-01T00:00:00.000,12.0


In [19]:
air_quality_df = air_quality_df.dropna(subset=['geo_join_id'])

In [21]:
air_quality_df['geo_join_id'] = air_quality_df['geo_join_id'].astype('int64')

In [27]:
air_quality_df['name'].unique()

array(['Ozone (O3)', 'Nitrogen dioxide (NO2)', 'Fine particles (PM 2.5)',
       'Annual vehicle miles traveled (cars)',
       'Annual vehicle miles traveled (trucks)',
       'Annual vehicle miles traveled',
       'Respiratory hospitalizations due to PM2.5 (age 20+)',
       'Asthma emergency department visits due to PM2.5',
       'Asthma emergency departments visits due to Ozone',
       'Cardiovascular hospitalizations due to PM2.5 (age 40+)',
       'Cardiac and respiratory deaths due to Ozone',
       'Asthma hospitalizations due to Ozone', 'Deaths due to PM2.5',
       'Boiler Emissions- Total PM2.5 Emissions',
       'Boiler Emissions- Total SO2 Emissions',
       'Boiler Emissions- Total NOx Emissions',
       'Outdoor Air Toxics - Formaldehyde',
       'Outdoor Air Toxics - Benzene'], dtype=object)

In [29]:
air_quality_df = air_quality_df[air_quality_df['name'].str.contains('Outdoor Air Toxics') | air_quality_df['name'].str.contains('Boiler Emissions') ]

In [39]:
simplified_tree_df = tree_census_df[['the_geom', 'zipcode', 'zip_city', 'cb_num', 'borocode', 'boroname', 'boro_ct', 'state']]

In [49]:
simplified_tree_df = simplified_tree_df[(simplified_tree_df['zipcode'] >= 10000) & (simplified_tree_df['zipcode']<11500)]
simplified_tree_df = simplified_tree_df.rename(columns={'the_geom': 'geometry', 'oldName2': 'newName2'})

In [55]:
simplified_tree_df.shape

(670351, 8)

In [57]:
simplified_tree_df = simplified_tree_df.drop_duplicates(subset=['cb_num'])

In [59]:
simplified_tree_df.shape

(59, 8)

In [71]:
air_quality_geo_df =  air_quality_df.merge(simplified_tree_df, left_on='geo_join_id', right_on='cb_num')

In [75]:
air_quality_geo_df.head()

,Unnamed: 0,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,the_geom,zipcode,zip_city,cb_num,borocode,boroname,boro_ct,state
0,8151,179825,641,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,406,Fresh Meadows,2015,2015-01-01T00:00:00.000,0.4,"{'type': 'Point', 'coordinates': [-73.84421521...",11375,Forest Hills,406,4,Queens,4073900,New York
1,8157,179783,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,104,Pelham - Throgs Neck,2015,2015-01-01T00:00:00.000,2.8,"{'type': 'Point', 'coordinates': [-73.98729652...",10019,New York,104,1,Manhattan,1012700,New York
2,8158,179773,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,404,Bayside - Little Neck,2015,2015-01-01T00:00:00.000,0.9,"{'type': 'Point', 'coordinates': [-73.86529991...",11373,Elmhurst,404,4,Queens,4044302,New York
3,8159,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01T00:00:00.000,6.1,"{'type': 'Point', 'coordinates': [-73.84344476...",11414,Howard Beach,410,4,Queens,4089200,New York
4,8160,179792,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,206,Borough Park,2015,2015-01-01T00:00:00.000,1.1,"{'type': 'Point', 'coordinates': [-73.89338225...",10457,Bronx,206,2,Bronx,2037504,New York


# Green Streets Data Sanitation

In [25]:
green_streets_df.shape

(2753, 26)

# Property Valuation Data Sanitation

In [26]:
property_valuation_df.shape

(1100000, 45)